In [ ]:
# server.py

import flwr as fl
from typing import Dict, List, Tuple
import torch
from ultralytics import YOLO
import warnings
from flwr.common import parameters_to_ndarrays
from collections import OrderedDict
from flwr.server.strategy import DPFedAvgFixed, FedAvg, DPFedAvgAdaptive



# ✅ Global model
model = YOLO(r"E:\PFE\Flower code\yolo models\yolo11m_mass.pt", task="detect")
model.model.nc = 2
model.model.names = {0: "no_mass", 1: "mass"}
model.fuse()
model.model = model.model.to("cuda")


# ✅ Path to validation YAML (use any one full dataset)
VAL_YAML = r"E:\PFE\Flower code\data created\client_cbis_ddsm0\cbis_ddsm.yaml"


# ✅ Helper: Set weights
def set_weights(weights):
    state_dict = OrderedDict({
        k: torch.tensor(v) for k, v in zip(model.model.state_dict().keys(), weights)
    })
    with torch.inference_mode():
        model.model.load_state_dict(state_dict, strict=True)



# ✅ Helper: Get weights
def get_weights():
    return [val.cpu().numpy() for _, val in model.model.named_parameters()]

# ✅ Helper: Save model
def save_model(round_num):
    model_path = fr"E:\PFE\Flower code\yolo models\global_model_round_{round_num}.pt"
    model.save(model_path)
    print(f"💾 Saved global model at: {model_path}")



# ✅ Custom strategy
class YOLOStrategy(FedAvg):
    def configure_evaluate(self, server_round, parameters, client_manager):
        return []

    def evaluate(self, rnd, parameters):
        return None



    def aggregate_fit(
        self,
        rnd: int,
        results: List[Tuple[fl.server.client_proxy.ClientProxy, fl.common.FitRes]],
        failures: List[BaseException]
    ) -> Tuple[List[torch.Tensor], Dict]:
        aggregated_weights, metrics = super().aggregate_fit(rnd, results, failures)

        if aggregated_weights is not None:
            weights_list = parameters_to_ndarrays(aggregated_weights)
            set_weights(weights_list)

            # ✅ Save global model
            save_model(rnd)

            # ✅ Evaluate on central validation and train set
            metrics_val = model.val(data=VAL_YAML, split="val")
            metrics_train = model.val(data=VAL_YAML, split="train")

            print(f"\n🌍 [Global model after round {rnd}]")
            print(f"Train:  mAP50={metrics_train.box.map50:.4f}, Recall={metrics_train.box.mr:.4f}")
            print(f"Val:    mAP50={metrics_val.box.map50:.4f}, Recall={metrics_val.box.mr:.4f}\n")

        return aggregated_weights, metrics or {}
    

base_strategy = YOLOStrategy(
fraction_fit=1.0,
min_fit_clients=11,
min_available_clients=11,
)
strategy = DPFedAvgFixed(strategy=base_strategy,
                         noise_multiplier=0.5,
                         clip_norm = 1.0,
                         num_sampled_clients=11
                        )


# ✅ Start server
fl.server.start_server(
    server_address="localhost:9675",
    config=fl.server.ServerConfig(num_rounds=100, round_timeout= None),
    strategy = strategy
)


YOLO11m summary (fused): 125 layers, 20,031,574 parameters, 0 gradients, 67.7 GFLOPs



            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
	Instead, use the `flower-superlink` CLI command to start a SuperLink as shown below:

		$ flower-superlink --insecure

	To view usage and all available options, run:

		$ flower-superlink --help

	Using `start_server()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower server, config: num_rounds=100, no round_timeout
INFO :      Flower ECE: gRPC server running (100 rounds), SSL is disabled
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
INFO :      Received initial parameters from one random client
INFO :      Starting evaluation of initial global parameters
INFO :      Evaluation returned no results (`None`)
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 2 clients (out of 2)
INFO :   

💾 Saved global model at: E:\PFE\Flower code\yolo models\global_model_round_1.pt
Ultralytics 8.3.107  Python-3.11.9 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce GTX 1660 SUPER, 6144MiB)


val: Scanning E:\PFE\Flower code\data created\client_cbis_ddsm0\valid\labels.cache... 5 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5/5 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:11<00:00, 11.14s/it]


                   all          5          5          0          0          0          0
               no_mass          5          5          0          0          0          0
Speed: 2.6ms preprocess, 103.7ms inference, 0.0ms loss, 122.5ms postprocess per image
Results saved to runs\detect\val24
Ultralytics 8.3.107  Python-3.11.9 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce GTX 1660 SUPER, 6144MiB)


...

INFO :      configure_evaluate: no clients selected, skipping evaluation
INFO :      
INFO :      [ROUND 3]
INFO :      configure_fit: strategy sampled 2 clients (out of 2)
INFO :      aggregate_fit: received 2 results and 0 failures

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        


💾 Saved global model at: E:\PFE\Flower code\yolo models\global_model_round_3.pt
Ultralytics 8.3.107  Python-3.11.9 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce GTX 1660 SUPER, 6144MiB)


val: Scanning E:\PFE\Flower code\data created\client_cbis_ddsm0\valid\labels.cache... 5 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5/5 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:13<00:00, 13.99s/it]
Exception in thread Thread-25 (plot_images):
Traceback (most recent call last):
  File "c:\Users\never\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "C:\Users\never\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\never\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\never\AppData\Local\Programs\Python\Python311\Lib\site-packages\ultralytics\utils\plotting.py", line 747, in plot_images
    annotator.box_label(box, label, color=color, rotated=is_ob

                   all          5          5          0          0          0          0
               no_mass          5          5          0          0          0          0
Speed: 35.7ms preprocess, 40.2ms inference, 0.0ms loss, 85.1ms postprocess per image
Results saved to runs\detect\val28
Ultralytics 8.3.107  Python-3.11.9 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce GTX 1660 SUPER, 6144MiB)


val: Scanning E:\PFE\Flower code\data created\client_cbis_ddsm0\train\labels.cache... 4 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4/4 [00:00<?, ?it/s]
